In [1]:
import gradio as gr
from sklearn.preprocessing import StandardScaler
import pandas as pd
import joblib
import pyodbc
import warnings

warnings.filterwarnings('ignore') 
pd.set_option('display.max_columns', 500)

scaler = StandardScaler()

model = joblib.load('xgb_best_model_task1.pkl')
model_kmeans = joblib.load('kmeans_model_new.pkl')
vectorizer = joblib.load('vectorizer_task1.pkl')

C:\Users\User\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 1) Load the Data Features from SQL server

In [2]:
details = ('DRIVER={SQL SERVER};'
           'SERVER=DESKTOP-OEN5051\SQLEXPRESS;'
           'DATABASE=Bank Loan;'
           'Trusted_Connection= yes;')

connection = pyodbc.connect(details)
cursor = connection.cursor()

In [3]:
sql_querys = "SELECT * FROM [Bank Loan].[dbo].[ml_testing_table1]"

df= pd.read_sql_query(sql_querys, connection)
df.shape

(38571, 16)

In [4]:
df['loan_amount'].max()

35000.0

In [5]:
numerical = ['annual_income', 'dti', 'installment', 'int_rate', 'loan_amount', 'total_acc','total_payment']

In [6]:
scaler = StandardScaler()
scaled_df = scaler.fit_transform(df[numerical])

In [7]:
def mapping(x):
    if x == '10 > years':
        return '11 years'
    if x == '< 1 year':
        return '0 year'
    else:
        return x
    
df['emp_length'] = df['emp_length'].apply(mapping)

In [8]:
df['job_title'] = df['job_title'].fillna('None')
categorical = ['emp_length', 'term', 'grade', 'purpose', 'job_title', 'issue_date_month', 'address_state_full_update', 'sub_grade','home_ownership']
df_X = pd.get_dummies(df, columns=categorical, dtype=int)
df_X = df_X.drop(columns = ['home_ownership_None'], axis = 1)

In [9]:
df_X

,annual_income,dti,installment,int_rate,loan_amount,total_acc,total_payment,emp_length_0 year,emp_length_1 year,emp_length_11 years,emp_length_2 years,emp_length_3 years,emp_length_4 years,emp_length_5 years,emp_length_6 years,emp_length_7 years,emp_length_8 years,emp_length_9 years,term_36 months,term_60 months,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,purpose_Car,purpose_Credit card,purpose_Debt consolidation,purpose_Educational,purpose_Home improvement,purpose_House,purpose_Major purchase,purpose_Medical,purpose_Moving,purpose_Other,purpose_Renewable_energy,purpose_Small business,purpose_Vacation,purpose_Wedding,job_title_Business,job_title_Corporations,job_title_Education,job_title_Facilities,job_title_Financial Services,job_title_Healthcare,job_title_Insurance Services,job_title_Military,job_title_None,job_title_Organizations,job_title_Public Service,job_title_Retail,job_title_Self-Employed,job_title_Service Providers,issue_date_month_April,issue_date_month_August,issue_date_month_December,issue_date_month_February,issue_date_month_January,issue_date_month_July,issue_date_month_June,issue_date_month_March,issue_date_month_May,issue_date_month_November,issue_date_month_October,issue_date_month_September,address_state_full_update_Alabama,address_state_full_update_Alaska,address_state_full_update_Arizona,address_state_full_update_Arkansas,address_state_full_update_California,address_state_full_update_Colorado,address_state_full_update_Connecticut,address_state_full_update_Delaware,address_state_full_update_District of Columbia,address_state_full_update_Florida,address_state_full_update_Georgia,address_state_full_update_Hawaii,address_state_full_update_Idaho,address_state_full_update_Illinois,address_state_full_update_Indiana,address_state_full_update_Iowa,address_state_full_update_Kansas,address_state_full_update_Kentucky,address_state_full_update_Louisiana,address_state_full_update_Maine,address_state_full_update_Maryland,address_state_full_update_Massachusetts,address_state_full_update_Michigan,address_state_full_update_Minnesota,address_state_full_update_Missouri,address_state_full_update_Montana,address_state_full_update_Nebraska,address_state_full_update_Nevada,address_state_full_update_New Hampshire,address_state_full_update_New Jersey,address_state_full_update_New Mexico,address_state_full_update_New York,address_state_full_update_North Carolina,address_state_full_update_Ohio,address_state_full_update_Oregon,address_state_full_update_Others,address_state_full_update_Pennsylvania,address_state_full_update_Rhode Island,address_state_full_update_South Carolina,address_state_full_update_Texas,address_state_full_update_Utah,address_state_full_update_Virginia,address_state_full_update_Washington,address_state_full_update_West Virginia,address_state_full_update_Wisconsin,address_state_full_update_Wyoming,sub_grade_A1,sub_grade_A2,sub_grade_A3,sub_grade_A4,sub_grade_A5,sub_grade_B1,sub_grade_B2,sub_grade_B3,sub_grade_B4,sub_grade_B5,sub_grade_C1,sub_grade_C2,sub_grade_C3,sub_grade_C4,sub_grade_C5,sub_grade_D1,sub_grade_D2,sub_grade_D3,sub_grade_D4,sub_grade_D5,sub_grade_E1,sub_grade_E2,sub_grade_E3,sub_grade_E4,sub_grade_E5,sub_grade_F1,sub_grade_F2,sub_grade_F3,sub_grade_F4,sub_grade_F5,sub_grade_G1,sub_grade_G2,sub_grade_G3,sub_grade_G4,sub_grade_G5,home_ownership_Mortgage,home_ownership_Other,home_ownership_Own,home_ownership_Rent
0,30000.0,0.0100,59.830002,0.1527,2500.0,4,1009,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,48000.0,0.0535,109.430000,0.1864,3000.0,4,3939,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [10]:


def predict_loan_approval(annual_income, dti, installment, int_rate, loan_amount, total_acc, total_payment, emp_length,term, grade,purpose, job_title, issue_date_month, address_state_full_update,
                           sub_grade,home_ownership):
    ###### DataFrame ################## 
    df_test = pd.DataFrame({"annual_income": [annual_income],
                        "dti": [dti],
                        "installment": [installment],
                        "int_rate": [int_rate],
                        "loan_amount": [loan_amount],
                        "total_acc": [total_acc],
                        "total_payment": [total_payment],
                        "emp_length": [emp_length],
                        "term": [term],
                        "grade": [grade],
                        "purpose": [purpose],
                        "job_title": [job_title],
                        "issue_date_month": [issue_date_month],
                        "address_state_full_update": [address_state_full_update],
                        "sub_grade": [sub_grade],
                        "home_ownership": [home_ownership]
    })
    df_test['sub_grade']  = df_test['grade'] + df_test['sub_grade'] 
    numerical = ['annual_income', 'dti', 'installment', 'int_rate', 'loan_amount', 'total_acc','total_payment']
    df_test['emp_length'] = df_test['emp_length'].apply(mapping)
    test_encoded = pd.get_dummies(df_test, columns=categorical, dtype=int)
    test_encoded = test_encoded.reindex(columns=df_X.columns)
    
    test_encoded_numerical = test_encoded[numerical]
    
    
    test_encoded_scaled = scaler.transform(test_encoded_numerical)
    
    test_encoded[numerical] = test_encoded_scaled
    prediction = model.predict(test_encoded)

    if prediction == 1:
        return "Approved"
    else:
        return "Not Approved"

Emp_length_options = ['< 1 year', '1 year', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', '8 years', '9 years', '10 years', '10 years >']
Term_options = ['36 months', '60 months']
Grade_options = ['A', 'B', 'C', 'D', 'E', 'F', 'G']
Purpose_options = ['Debt consolidation', 'Credit card', 'Car', 'Home improvement','Small business', 'Major purchase', 'Moving', 'Other', 'House','Vacation', 'Educational', 'Wedding', 'Medical','Renewable_energy']
Job_Title_options = ['Business', 'Corporations', 'Education', 'Facilities', 'Financial Services', 'Healthcare', 'Insurance Services', 'Military', 'None', 'Organizations', 'Public Service', 'Retail', 'Self-Employed', 'Service Providers']
Issue_Date_month_options = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September', 'October', 'November', 'December']
address_state_options = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 
                         'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 
                         'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 
                         'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 
                         'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 
                         'Wisconsin', 'Wyoming', 'Others']

sub_grade_options = ['1','2','3','4','5']

home_ownership_options = ['Rent', 'Own', 'Mortgage']

iface = gr.Interface(
    fn=predict_loan_approval,
    inputs=[
        gr.Slider(0, 3000000, label="Annual Income (USD)"),
        gr.Slider(0, 0.3, label="Debt-to-Income Ratio"),
        gr.Slider(0, 1500, label="Monthly Installment (USD)"),
        gr.Slider(0, 0.25, label="Interest Rate"),
        gr.Slider(0, 100000, label="Loan Amount (USD)"),
        gr.Slider(0, 100, label="Total Number of Accounts"),
        gr.Slider(0, 40000, label="Total Payment (USD)"),
        gr.Dropdown(choices=Emp_length_options, label="Employment Length"),
        gr.Dropdown(choices=Term_options, label="Loan Term"),
        gr.Dropdown(choices=Grade_options, label="Loan Grade"),
        gr.Dropdown(choices=Purpose_options, label="Loan Purpose"),
        gr.Dropdown(choices=Job_Title_options, label="Employment Sector"),
        gr.Dropdown(choices=Issue_Date_month_options, label="Issue Date Month"),
        gr.Dropdown(choices=address_state_options, label="State"),
        gr.Dropdown(choices=sub_grade_options, label="Loan Sub Grade"),
        gr.Dropdown(choices=home_ownership_options, label="Home Ownership"),
    ],
    outputs="text",
    title="Loan Approval Predictor",
    description="Please enter your financial and personal details to predict the likelihood of loan approval.",
   
)

# Launch the interface
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
